# Imports and settings

In [18]:
from openmm.app import *
from openmm import *                    
from openmm.unit import *
from openmmtools import forces
import mdtraj as md
import numpy as np
import parmed as pmd
import bz2
import os
from openmm import CustomIntegrator
from openmm.unit import kilojoules_per_mole, is_quantity
from openmm.unit import *
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

In [19]:
# define a function for creating RMSD restraints 
def create_rmsd_restraint(positions, atom_indicies):
    rmsd_cv = RMSDForce(positions, atom_indicies)
    energy_expression = 'step(dRMSD) * (K_RMSD/2) * dRMSD^2; dRMSD = (RMSD-RMSD0);'                                                  
    energy_expression += 'K_RMSD = %f;' % spring_constant.value_in_unit_system(md_unit_system)                                       
    energy_expression += 'RMSD0 = %f;' % restraint_distance.value_in_unit_system(md_unit_system)                                     
    restraint_force = CustomCVForce(energy_expression)                                                                               
    restraint_force.addCollectiveVariable('RMSD', rmsd_cv)                                                                           
    return restraint_force                                                                                                           
                                                                  

In [20]:
# define a function to list force groups and modify system context                                                       
def forcegroupify(system):                                        
    forcegroups = {}                                              
    for i in range(system.getNumForces()):                                                                                           
        force = system.getForce(i)                           
        force.setForceGroup(i) 
        forcegroups[force] = i                                    
    return forcegroups

# Set up simulation + state

In [21]:
sim_temp = 300.0 * kelvin
H_mass = 4.0 * amu #Might need to be tuned to 3.5 amu 
time_step = 0.002 * picosecond  
nb_cutoff = 10.0 * angstrom                                                                                                          
box_padding = 12.0 * angstrom
salt_conc = 0.15 * molar
receptor_path="../villin.pdb"
current_file="villin-solvated"
# Misc parameters                                                 
restraint_distance = 0.0 * angstroms                              
restart_freq = 10
log_freq = 1                                                                                                                         
prd_steps = 100     

In [22]:
## Load an already solvated PDB file and set up the system + state
pdb = PDBFile(receptor_path)
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology, 
                                         nonbondedMethod=PME, 
                                         nonbondedCutoff=nb_cutoff, 
                                         constraints=HBonds, 
                                         rigidWater=True,
                                         hydrogenMass=H_mass)
system.addForce(MonteCarloBarostat(1*bar, sim_temp))


5

# Identify and setup `CVForce`sets

In [23]:
prot_top = md.Topology.from_openmm(pdb.topology)
# find two atoms and save their inds
d1_atom1_ind = prot_top.select("residue 6 and name CZ")[0]
d1_atom2_ind = prot_top.select("residue 10 and name CZ")[0] 
d2_atom2_ind = prot_top.select("residue 17 and name CZ")[0]

In [24]:
# Let's define a torsion we will use later
phe_chi = [72, 74, 76, 79]

## Let's load in D1 and D2 from an existing langevin dynamics log and decide on parameters

In [ ]:
columts = ['step', 'd1', 'd2', 'NaN']
cv_eq = pd.read_csv("./meta-start-sample.dat",
                    delimiter=" ",
                   names=columts)

d1_eq = cv_eq['d1'].values
d2_eq = cv_eq['d2'].values

In [ ]:
import matplotlib.pyplot as plt
plt.hist(d1_eq)
print("average: %s \n error %s" % (np.mean(d1_eq), np.std(d1_eq)))
print("error value should thus be %s" % (np.std(d1_eq) / 2))

In [ ]:
plt.hist(d2_eq)
print("average: %s \n error %s" % (np.mean(d2_eq), np.std(d2_eq)))
print("error value should thus be %s" % (np.std(d2_eq) / 2))

# Setup a fresh MetaD simulation

In [25]:
# Define BiasVariables - Note that I've set the distances to be "Periodic=True"
# This allows us to use Mixed periodic and Non-periodic variables
chi = CustomTorsionForce('theta')
chi.addTorsion(phe_chi[0], phe_chi[1], phe_chi[2], phe_chi[3])
chi_bias = BiasVariable(chi, -np.pi, np.pi, 0.35, True, 181)

# Periodic dists
d1 = CustomBondForce('r')
d1.addBond(d1_atom1_ind, d1_atom2_ind)
d1_bias = BiasVariable(d1, 0.3, 5.0, 0.05, True)

d2=CustomBondForce('r')
d2.addBond(d1_atom1_ind, d2_atom2_ind)
d2_bias = BiasVariable(d2, 0.3, 5.0, 0.05, True)


In [26]:
# the meta object
meta = Metadynamics(system, [d1_bias, d2_bias], 300, 
                    biasFactor=5, height=1, frequency=1, 
                    saveFrequency=10, 
                    biasDir="./")

In [27]:
#Define an integrator
integrator = LangevinIntegrator(sim_temp, 1.0/picosecond, time_step)

In [28]:
# SEtup platform
platform = openmm.Platform.getPlatformByName('OpenCL')
platform.setPropertyDefaultValue('Precision', 'mixed')
simulation = Simulation(pdb.topology, system, integrator, platform)

In [30]:
current_file = "test-meta"
simulation.reporters.append(DCDReporter('./'+current_file+''+ '.dcd', restart_freq))
simulation.reporters.append(CheckpointReporter('./'+current_file+''+ '.chk', min(prd_steps, 10*restart_freq)))
simulation.reporters.append(StateDataReporter(open('./log.' + current_file+'', 'w'), log_freq, step=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, speed=True))


In [31]:
simulation.context.setPositions(pdb.positions)
print('  initial : %s' % (simulation.context.getState(getEnergy=True).getPotentialEnergy()))
simulation.minimizeEnergy()
print('  final : %s' % (simulation.context.getState(getEnergy=True).getPotentialEnergy()))

  initial : -115004.63008636737 kJ/mol
  final : -145453.11768340121 kJ/mol


In [33]:
steps=5
with open("./try3-cv-tracker.dat", "a") as f:
    for i in range(steps):
        meta.step(simulation, steps=10)
        n_steps = str(simulation.context.getStepCount())
        d1,d2 =meta._force.getCollectiveVariableValues(simulation.context)
        f.write("%s %s %s \n" % (n_steps, str(d1), str(d2)))
        meta._syncWithDisk()
        print(meta.getFreeEnergy())
print("Done with simulation.")

[[-6.65667027e-04 -1.10146234e-03 -1.75080448e-03 ... -2.15525171e-04
  -3.86455797e-04 -6.65667027e-04]
 [-1.31279319e-03 -2.17224603e-03 -3.45284511e-03 ... -4.25047136e-04
  -7.62147466e-04 -1.31279319e-03]
 [-2.48707995e-03 -4.11530976e-03 -6.54139992e-03 ... -8.05249289e-04
  -1.44388416e-03 -2.48707995e-03]
 ...
 [-1.51719351e-04 -2.51046061e-04 -3.99044411e-04 ... -4.91227057e-05
  -8.80813440e-05 -1.51719351e-04]
 [-3.24243988e-04 -5.36518193e-04 -8.52810153e-04 ... -1.04981569e-04
  -1.88241251e-04 -3.24243988e-04]
 [-6.65667027e-04 -1.10146234e-03 -1.75080448e-03 ... -2.15525171e-04
  -3.86455797e-04 -6.65667027e-04]] kJ/mol


Exception ignored in: <function DCDReporter.__del__ at 0x11b4bf8b0>
Traceback (most recent call last):
  File "/Users/singhs15/miniconda3/envs/openmm/lib/python3.9/site-packages/openmm/app/dcdreporter.py", line 110, in __del__
    self._out.close()
AttributeError: 'DCDReporter' object has no attribute '_out'


[[-1.06828420e-03 -1.76602053e-03 -2.80453097e-03 ... -3.46525792e-04
  -6.20773845e-04 -1.06828420e-03]
 [-2.10556557e-03 -3.48079279e-03 -5.52768291e-03 ... -6.82993570e-04
  -1.22353083e-03 -2.10556557e-03]
 [-3.98664220e-03 -6.59048103e-03 -1.04660431e-02 ... -1.29316576e-03
  -2.31661015e-03 -3.98664220e-03]
 ...
 [-2.43776869e-04 -4.02995791e-04 -6.39976573e-04 ... -7.90757112e-05
  -1.41657648e-04 -2.43776869e-04]
 [-5.20668586e-04 -8.60735781e-04 -1.36689101e-03 ... -1.68892770e-04
  -3.02557862e-04 -5.20668586e-04]
 [-1.06828420e-03 -1.76602053e-03 -2.80453097e-03 ... -3.46525792e-04
  -6.20773845e-04 -1.06828420e-03]] kJ/mol
[[-0.00140648 -0.00232536 -0.00369321 ... -0.00045613 -0.00081721
  -0.00140648]
 [-0.0027639  -0.00456961 -0.00725758 ... -0.00089635 -0.00160591
  -0.0027639 ]
 [-0.00521773 -0.00862655 -0.01370089 ... -0.00169214 -0.00303167
  -0.00521773]
 ...
 [-0.00032289 -0.00053385 -0.00084788 ... -0.00010472 -0.00018761
  -0.00032289]
 [-0.00068756 -0.00113677 -0